<a href="https://colab.research.google.com/github/shayesteh99/Proteome-Prediction/blob/main/fully_connected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials# Authenticate and create the PyDrive client.
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id = "1-f4bTkNO4VI7yjDmipTySOGL18KA3-aI"
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('gene.train.csv')
gene1 = pd.read_csv('gene.train.csv')

In [ ]:
gene_train = gene1.drop(columns = ["Cancer_Type", "Sample_ID"])

In [ ]:
gene_train.head

<bound method NDFrame.head of       ENSG00000141510  ENSG00000155657  ...  ENSG00000277957  ENSG00000258453
0            3.038108         0.060619  ...         0.032929              0.0
1            3.346721         0.052940  ...         0.095788              0.0
2            2.043045         0.194234  ...         0.015251              0.0
3            3.139093         0.120746  ...         0.000000              0.0
4            3.574059         0.089624  ...         0.021285              0.0
...               ...              ...  ...              ...              ...
5733         4.899541         0.144547  ...         0.045017              0.0
5734         4.481275         0.248651  ...         0.008160              0.0
5735         4.588105         0.021127  ...         0.014531              0.0
5736         4.457775         0.022709  ...         0.027554              0.0
5737         4.980895         0.122636  ...         0.042833              0.0

[5738 rows x 19420 columns]>

In [ ]:
id = "1Yg4vE4DCvVxQQ3lftzAf-OcFiV3K0InM"
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('gene.valid.csv')
gene2 = pd.read_csv('gene.valid.csv')

In [ ]:
gene_test = gene2.drop(columns = ["Cancer_Type", "Sample_ID"])

In [ ]:
id = '1kHr-g60285ylV5VNJQpAnqfXWJC9pWUy'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('pro.train.csv')
pro1 = pd.read_csv('pro.train.csv')

In [ ]:
lable_train = list(pro1.Cancer_Type)
pro_train = pro1.drop(columns = ["Cancer_Type", "Sample_ID"])
print(len(set(lable_train)))

32


In [ ]:
id = '1JHJEPZi5OJkMjieZT8rcSb9W8bzRPQbg'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('pro.valid.csv')
pro2 = pd.read_csv('pro.valid.csv')

In [ ]:
lable_test = list(pro2.Cancer_Type)
pro_test = pro2.drop(columns = ["Cancer_Type", "Sample_ID"])

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

le = LabelEncoder()
lable1 = np.ravel(lable_train)
le.fit(lable1)
lable1 = le.transform(lable1)

le = LabelEncoder()
lable2 = np.ravel(lable_test)
le.fit(lable2)
lable2 = le.transform(lable2)


In [ ]:
from keras.utils import np_utils
cat1 = np_utils.to_categorical(lable1)
cat1 = pd.DataFrame({'tag' + str(i) : cat1[:, i]for i in range(len(cat1[0]))})
cat_train = cat1[:gene_train.shape[0]]
print(cat_train)

cat2 = np_utils.to_categorical(lable2)
cat2 = pd.DataFrame({'tag' + str(i) : cat2[:, i]for i in range(len(cat2[0]))})
cat_test = cat2[:gene_test.shape[0]]
print(cat_test)

      tag0  tag1  tag2  tag3  tag4  ...  tag27  tag28  tag29  tag30  tag31
0      1.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    0.0
1      1.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    0.0
2      1.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    0.0
3      1.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    0.0
4      1.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    0.0
...    ...   ...   ...   ...   ...  ...    ...    ...    ...    ...    ...
5733   0.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    1.0
5734   0.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    1.0
5735   0.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    1.0
5736   0.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    1.0
5737   0.0   0.0   0.0   0.0   0.0  ...    0.0    0.0    0.0    0.0    1.0

[5738 rows x 32 columns]
     tag0  tag1  tag2  tag3  tag4  ...  tag27  tag28  tag29  tag30  tag31


In [ ]:
from keras.layers import Input, Dense, BatchNormalization, Lambda, GaussianNoise, GaussianDropout, Add
from keras.models import Model
import keras.backend as backend
import numpy as np
import keras
from sklearn.preprocessing import LabelEncoder, normalize
from keras.utils import np_utils
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
with tf.Session() as sess:
     sess.run(tf.global_variables_initializer())
import time

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
def model(m_rna_train, pro_train,label_train, m_rna_test, pro_test, label_test):
        opt = keras.optimizers.Nadam(learning_rate=1e-3)

        inputs = Input(shape=(m_rna_train.shape[1],), name="inputs")
        inputs_noise = GaussianNoise(stddev=0.25)(inputs)
        inputs_noise = GaussianDropout(rate=0.5 ** 2 / (1 + 0.5 ** 2))(inputs_noise)
        inputs_0 = BatchNormalization(name="inputs_0")(inputs_noise)
        inputs_1 = Dense(2048, activation="relu", name="inputs_1")(inputs_0)
        inputs_2 = BatchNormalization(name="inputs_2")(inputs_1)
        drop1 = GaussianDropout(rate=0.5 ** 2 / (1 + 0.5 ** 2))(inputs_2)
        inputs_3 = Dense(1024, activation="relu", name="inputs_3")(drop1)
        inputs_4 = BatchNormalization(name="inputs_4")(inputs_3)
        drop2 = GaussianDropout(rate=0.5 ** 2 / (1 + 0.5 ** 2))(inputs_4)
        inputs_5 = Dense(512, activation="relu", name="inputs_5")(drop2)
        inputs_6 = BatchNormalization(name="inputs_6")(inputs_5)
        drop3 = GaussianDropout(rate=0.5 ** 2 / (1 + 0.5 ** 2))(inputs_6)
        inputs_7 = Dense(256, activation="relu", name="inputs_7")(drop3)
        inputs_8 = BatchNormalization(name="inputs_8")(inputs_7)
        drop4 = GaussianDropout(rate=0.5 ** 2 / (1 + 0.5 ** 2))(inputs_8)


        decoded_micro_rna = Dense(units=pro_train.shape[1], activation='linear', name="pro")(drop4)

        scae = Model(inputs=inputs, outputs=[decoded_micro_rna])
        lambda_value = 2.7849e-3

        scae.compile(optimizer=opt,
                     loss=["mse"])
        
        history = scae.fit([m_rna_train], [pro_train],
                 batch_size=128, epochs=200, verbose=1, shuffle = True)
        score = scae.evaluate(m_rna_test, [pro_test], verbose=1,
                              batch_size=128)
        pred_pro = scae.predict(m_rna_test, verbose=1,
                              batch_size=128)
        pred_train = scae.predict(m_rna_train, verbose=1,
                              batch_size=128)
        return pred_train, pred_pro, score, history


In [ ]:
print(gene_train.shape, gene_test.shape)
print(pro_train.shape, pro_test.shape)

(5738, 19420) (673, 19420)
(5738, 216) (673, 216)


In [ ]:
import math

train_y = pro_train.to_numpy()
test_y = pro_test.to_numpy()
label_train = np.array(cat_train)
label_test = np.array(cat_test)

train_x_log=np.log2(gene_train)
train_x_log1=train_x_log.replace(to_replace = np.nan, value =0.00)
train_x_log2=train_x_log1.replace(to_replace =-math.inf, value =0.0000)
arr=[]
for i in np.arange(train_x_log2.shape[1]):
  idx=(train_x_log2.iloc[:,i]==0.0)
  arr.append(len(train_x_log2.iloc[:,i][idx]))
arr_idx=[]
for i in np.arange(train_x_log2.shape[1]):
  if arr[i]<=3000:
    arr_idx.append(train_x_log2.columns[i])
train_x_log3=train_x_log2[arr_idx]
val_x_log=np.log2(gene_test)
val_x_log1=val_x_log.replace(to_replace =-math.inf, value =0.0000)
val_x_log2=val_x_log1.replace(to_replace = np.nan, value =0.00)
val_x_log3=val_x_log2[arr_idx]
val_x_log3 = val_x_log3
print(val_x_log3.shape)
print(train_x_log3.shape)
print(test_y.shape)
print(train_y.shape)
pred_train, pred_pro, score, history = model(train_x_log3, train_y, label_train, val_x_log3, test_y, label_test)

print(score)
print(len(gene_test))

In [ ]:
from sklearn.metrics import mean_squared_error,median_absolute_error,mean_absolute_error,explained_variance_score,r2_score
from math import sqrt
print("explained_variance_score",explained_variance_score(test_y,pred_pro))
print("median_absolute_error",median_absolute_error(test_y,pred_pro))
print("r2_score",r2_score(test_y,pred_pro))
print("RMSE",sqrt(mean_squared_error(test_y,pred_pro)))
print("mean_absolute_error",mean_absolute_error(test_y,pred_pro))
print("mean_squared_error",mean_squared_error(test_y,pred_pro))

explained_variance_score 0.45825268163670685
median_absolute_error 0.20805213921870377
r2_score 0.45399121369580786
RMSE 0.434131167507341
mean_absolute_error 0.2817359740829059
mean_squared_error 0.18846987060128695


In [ ]:
def get_rankMean(test,predict):
    p = predict
    rank  = 0
    for k in range(len(p)):
        temp = np.vstack((test, p[k]))
        for i in range(temp.shape[0]):
            temp[i] = np.subtract(temp[i], test[k])
        temp = np.square(temp)
        sum = temp.sum(axis = 1)
        #print(sum[1558])
        arr = []
        for i in range(len(sum)):
            arr.append((sum[i], i))
        arr = sorted(arr, key=lambda x: x[0])
        for i in range(len(arr)):
            if arr[i][1] == len(p):
                rank += i
                break
    rank /= len(p)
    #len(p)
    return rank
val=test_y
predict1=pred_pro
get_rankMean(val,predict1)

2.6300148588410104

In [ ]:

# print("explained_variance_score",explained_variance_score(train_y,pred_train))
# print("median_absolute_error",median_absolute_error(train_y,pred_train))
print("r2_score",r2_score(train_y,pred_train))
# print("RMSE",sqrt(mean_squared_error(train_y,pred_train)))
# print("mean_absolute_error",mean_absolute_error(train_y,pred_train))
print("mean_squared_error",mean_squared_error(train_y,pred_train))

explained_variance_score 0.786911655634134
median_absolute_error 0.12227174561262541
r2_score 0.786783842186189
RMSE 0.21464224459061315
mean_absolute_error 0.15663965116354817
mean_squared_error 0.046071293162896605
